Defining a function for adding noise to the dataset for finetuning T5 correcting the spelling error

In [ ]:
%pip install transformers datasets --quiet
import random
from datasets import Dataset
df = pd.read_csv("/content/A_Z_medicines_dataset_of_India.csv")
def add_noise(text, noise_factor=0.4):
    """
    Add noisy OCR-like distortions to text.
    noise_factor controls how aggressive the noise is (0.1 = light, 0.4 = heavy)
    """
    substitutions = {
        "0": "O", "O": "0",
        "1": "I", "I": "1", "l": "I",
        "5": "S", "S": "5",
        "8": "B", "B": "8",
        "c": "e", "e": "c",
        "m": "nn", "nn": "m",
        "rn":"m","m":"rn"
    }

    text = list(text)
    num_noises = max(1, int(len(text) * noise_factor))

    for _ in range(num_noises):
        op = random.choice(["swap", "delete", "insert", "replace", "duplicate", "caseflip", "punctuate", "space"])
        i = random.randint(0, len(text) - 1)

        if op == "swap" and i < len(text) - 1:
            text[i], text[i+1] = text[i+1], text[i]

        elif op == "delete":
            text[i] = ""

        elif op == "insert":
            text.insert(i, random.choice("abcdefghijklmnopqrstuvwxyz"))

        elif op == "replace":
            char = text[i]
            text[i] = substitutions.get(char, random.choice("abcdefghijklmnopqrstuvwxyz"))

        elif op == "duplicate":
            text.insert(i, text[i])

        elif op == "caseflip":
            text[i] = text[i].upper() if text[i].islower() else text[i].lower()

        elif op == "punctuate":
            text.insert(i, random.choice([".", ",", "-", "_"]))

        elif op == "space":
            text.insert(i, " ")

    noisy_text = "".join(text)
    return noisy_text.strip()

  # Load clean names
clean_names = df['name'].dropna().unique().tolist()[:20000]  # limit for speed

# Create noisy variants
data = [{"input_text": add_noise(name.lower()), "target_text": name} for name in clean_names]

dataset = Dataset.from_list(data)
print("✅ Synthetic dataset created with", len(dataset), "samples")
dataset[200]



For training the ByT5 model for spelling error correction  

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Trainer, TrainingArguments

model_name = "google/byt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def preprocess_function(examples):
    model_inputs = tokenizer(examples["input_text"], max_length=64, truncation=True)
    labels = tokenizer(examples["target_text"], max_length=64, truncation=True).input_ids
    model_inputs["labels"] = labels
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["input_text", "target_text"])

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="./byt5-medicine-corrector",
    eval_strategy="no",
    learning_rate=1e-4,
    per_device_train_batch_size=7,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    save_strategy="epoch",
    # Disable fused optimizer which is not supported by XLA
    optim="adafactor",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()
trainer.save_model("./byt5-medicine-corrector")
print(" Model fine-tuned and saved as ./byt5-medicine-corrector")

Finetuning FlanT5 for explanation of drug

In [ ]:
%pip install rapidfuzz transformers torch --quiet

import pandas as pd
from rapidfuzz import process, fuzz
from transformers import pipeline


df1 = pd.read_csv("/content/Medicine_Details.csv")

# Clean column names
df1.columns = [c.strip().replace(" ", "_") for c in df1.columns]


def get_closest_medicine(query, df1, limit=3):
    names = df1["Medicine_Name"].tolist()
    results = process.extract(query, names, scorer=fuzz.WRatio, limit=limit)
    return results  # list of (match, score, index)


def generate_drug_explanation(drug_name, uses, side_effects, composition=None):
    prompt = (
        f"The medicine '{drug_name}' "
        f"is composed of {composition if composition else 'active ingredients'}. "
        f"It is mainly used for {uses}. "
        f"The medicine works by affecting the body in a way that helps treat the condition. "
        f"Common side effects may include {side_effects}. "
        f"This information is for educational purposes only, not a medical prescription."
    )
    return prompt



generator = pipeline("text2text-generation", model="google/flan-t5-base", max_new_tokens=80)

def make_readable_explanation(raw_text):
    result = generator(f"Rephrase this professionally and simply: {raw_text}")
    return result[0]['generated_text']


#defining function- From OCR Input to Output

def get_drug_info(ocr_name):
    matches = get_closest_medicine(ocr_name, df1)
    best_match, score, idx = matches[0]
    record = df1.iloc[idx]

    raw_explanation = generate_drug_explanation(
        drug_name=record["Medicine_Name"],
        uses=record["Uses"],
        side_effects=record["Side_effects"],
        composition=record["Composition"]
    )

    readable_output = make_readable_explanation(raw_explanation)

    return {
        "OCR Input": ocr_name,
        "Matched Medicine": record["Medicine_Name"],
        "Match Confidence": score,
        "Generated Explanation": readable_output
    }


# Test 

example_inputs = ["Augmntin 625", "Azithrol 500", "Aciloc", "Ascoril syrup"]

for drug in example_inputs:
    result = get_drug_info(drug)
    print("\n OCR Input:", result["OCR Input"])
    print(" Matched Medicine:", result["Matched Medicine"], f"({result['Match Confidence']:.1f}%)")
    print(" Explanation:", result["Generated Explanation"])


Actual code for OCR, spelling correction, fuzzy matching, result

In [ ]:

%pip install transformers datasets jiwer pillow torch torchvision torchaudio rapidfuzz --quiet
%pip install opencv-python --quiet


from google.colab import files
uploaded = files.upload()

# Get uploaded filename (in colab)
#if running in VS code, change it to the imgage path 
img_path = list(uploaded.keys())[0]
print(" Uploaded file:", img_path)


import cv2
from PIL import Image
import matplotlib.pyplot as plt

# Load with OpenCV
img = cv2.imread(img_path, cv2.IMREAD_COLOR)

if img is None:
    raise ValueError(" Error: Image not loaded. Check the filename!")

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#  (a) Apply CLAHE for better contrast in low-quality scans
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
gray = clahe.apply(gray)

#  (b) Remove noise with Gaussian blur
blur = cv2.GaussianBlur(gray, (5, 5), 0)

#  (c) Apply adaptive thresholding
thresh = cv2.adaptiveThreshold(
    blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2
)

#  (d) Apply morphological closing to connect broken characters
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

# Save preprocessed image for OCR
preprocessed_path = "preprocessed.jpg"
cv2.imwrite(preprocessed_path, thresh)

# Show original vs preprocessed
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.title("Original")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1,2,2)
plt.title("Preprocessed (Enhanced)")
plt.imshow(thresh, cmap="gray")
plt.axis("off")
plt.show()


# Load TrOCR model

import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-handwritten")

# Convert preprocessed image for OCR
img_pil_pre = Image.fromarray(cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB))
pixel_values_pre = processor(images=img_pil_pre, return_tensors="pt").pixel_values

# Run inference
with torch.no_grad():
    generated_ids_pre = model.generate(pixel_values_pre)

text_pre = processor.batch_decode(generated_ids_pre, skip_special_tokens=True)[0]
print("\n TrOCR Output (preprocessed image):", text_pre)

# Also test with ORIGINAL image
img_pil_orig = Image.open(img_path).convert("RGB")
pixel_values_orig = processor(images=img_pil_orig, return_tensors="pt").pixel_values
with torch.no_grad():
    generated_ids_orig = model.generate(pixel_values_orig)

text_orig = processor.batch_decode(generated_ids_orig, skip_special_tokens=True)[0]
print(" TrOCR Output (original image):", text_orig)


# Medicine name correction using dataset

import pandas as pd
from rapidfuzz import process, fuzz


dataset_path = "/content/A_Z_medicines_dataset_of_India.csv"

# Load dataset
df = pd.read_csv(dataset_path)
medicine_names = df['name'].dropna().tolist()

# Choose which OCR output to use (you can compare both)
ocr_output = text_pre.strip() if len(text_pre) > len(text_orig) else text_orig.strip()

# Use fuzzy matching to find best match
best_match = process.extractOne(
    ocr_output,
    medicine_names,
    scorer=fuzz.token_sort_ratio
)

if best_match:
    predicted_name, score, _ = best_match
    print("\n OCR Raw Output:", ocr_output)
    print(f" Matched Medicine before Correction: {predicted_name}")
    print(f" Match Confidence: {score:.2f}")
else:
    print(" No suitable match found in dataset.")

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load fine-tuned model
corrector_tokenizer = AutoTokenizer.from_pretrained("./byt5-medicine-corrector")
corrector_model = AutoModelForSeq2SeqLM.from_pretrained("./byt5-medicine-corrector")

def correct_text(input_text):
    inputs = corrector_tokenizer(input_text, return_tensors="pt")
    with torch.no_grad():
        outputs = corrector_model.generate(**inputs, max_length=64)
    return corrector_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Apply correction before fuzzy match
corrected_ocr = correct_text(ocr_output)
best_match = process.extractOne(corrected_ocr, medicine_names, scorer=fuzz.token_sort_ratio)

print("\n Raw OCR:", ocr_output)
print(" Corrected by ByT5:", corrected_ocr)
result = get_drug_info(corrected_ocr)
print("\n OCR Input:", result["OCR Input"])
print(" Matched Medicine:", result["Matched Medicine"], f"({result['Match Confidence']:.1f}%)")
print(" Explanation:", result["Generated Explanation"])
if best_match:
    print(f" Matched Medicine: {best_match[0]}")
    print(f" Confidence: {best_match[1]:.2f}")
else:
    print("No match found.")